# NFT Authenticity and Prediction using Sentiment Analysis and Deep Learning

## Imports

In [2]:
import json
import asyncio
import tqdm
import tqdm.asyncio
import nest_asyncio
import pandas as pd
nest_asyncio.apply()
import os
from dotenv import load_dotenv
load_dotenv()
from pprint import pprint
from datetime import datetime
from dateutil.relativedelta import relativedelta
from aiohttp import ClientSession, TCPConnector

## Data Collection

Sample Data

In [3]:
sample_slugs = ['evolved-apes-inc','oraclenft','boredapeyachtclub','neo-tokyo-identities','cool-cats-nft','cryptopunks','veefriends'] #'evolved-apes-inc','oraclenft','boredapeyachtclub','neo-tokyo-identities','cool-cats-nft','cryptopunks',
slug_date = {}
slug_price = {}

### OpenSea Metrics from Collection Slugs

Asynchronous Data Collection

In [4]:
async def fetchData(url,session):
    async with session.get(url) as response:
        try:
            response = await response.read()
            assert 'detail' not in json.loads(response.decode('UTF-8'))
            return response
        except:
            await asyncio.sleep(5)
            return await fetchData(url,session)

async def transactionScrape():
    tasks = []
    responses = []
    connector = TCPConnector(limit_per_host=1)
    url = "https://api.opensea.io/api/v1/events?collection_slug={slug}&only_opensea=false&event_type=successful&limit=300&occurred_after={start}&occurred_before={end}"
    headers={"Accept": "application/json", "X-API-KEY": os.getenv('OPENSEA_API_KEY')}
    async with ClientSession(connector=connector, headers=headers) as session:
        for i in range(len(sample_slugs)):
            start_date = datetime.fromisoformat(slug_date[sample_slugs[i]].split('T')[0]) - relativedelta(days=1)
            for j in range(14):
                start_date = start_date + relativedelta(days=1)
                end_date = start_date + relativedelta(days=1) 
                task = asyncio.ensure_future(fetchData(url.format(slug=sample_slugs[i],start=start_date.timestamp(), end=end_date.timestamp()),session))
                tasks.append(task)
        for f in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            responses.append(await f)
    for response in responses:
        response = json.loads(response.decode('utf8'))
        try:
            collection_slug = response['asset_events'][0]['collection_slug'] 
            for event in response['asset_events']:
                slug_price[collection_slug]['total_volume'] += float(event['total_price'])/1000000000000000000
                slug_price[collection_slug]['num_transactions'] += 1
        except:
            pass

async def collectionScrape():
    tasks = []
    responses = []
    connector = TCPConnector()
    url = "https://api.opensea.io/api/v1/collection/{}"
    async with ClientSession(connector=connector) as session:
        for i in range(len(sample_slugs)):
            task = asyncio.ensure_future(fetchData(url.format(sample_slugs[i]),session))
            tasks.append(task)
        for f in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            responses.append(await f)
    for response in responses:
        response = json.loads(response.decode('utf8'))
        name = response['collection']['name']
        floor = response['collection']['stats']['floor_price']
        volume = response['collection']['stats']['total_volume']
        date_created = response['collection']['created_date']
        slug_date[response['collection']['slug']] = date_created
        slug_price[response['collection']['slug']] = {'total_volume' : 0, 'num_transactions': 0 }
        print(f'{name}: Floor: {floor}ETH --- Total Volume: {volume}ETH --- Created {date_created}')


Run Collection Script

In [5]:
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(collectionScrape())
loop.run_until_complete(future)

100%|██████████| 7/7 [00:00<00:00, 12.24it/s]

Bored Ape Yacht Club: Floor: 51.98ETH --- Total Volume: 249891.69985241446ETH --- Created 2021-04-22T23:14:03.967121
Neo Tokyo Identities: Floor: 16.1ETH --- Total Volume: 11319.287364958145ETH --- Created 2021-10-04T15:14:32.376341
CryptoPunks: Floor: NoneETH --- Total Volume: 733785.5812492912ETH --- Created 2019-04-26T22:13:09.691572
Cool Cats NFT: Floor: 7.95ETH --- Total Volume: 58888.31137248874ETH --- Created 2021-06-27T09:03:35.403074
Evolved Apes Inc: Floor: 0.0025ETH --- Total Volume: 1156.4932641192624ETH --- Created 2021-09-23T12:30:03.410309
Oracle (official): Floor: 0.0ETH --- Total Volume: 27.59211999999997ETH --- Created 2021-09-22T10:53:26.948567
VeeFriends: Floor: 7.939ETH --- Total Volume: 32038.28050392832ETH --- Created 2021-05-11T18:22:19.398578


Run Transaction Collections

In [6]:
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(transactionScrape())
loop.run_until_complete(future)
pprint(slug_price)

100%|██████████| 98/98 [06:52<00:00,  4.21s/it]


{'boredapeyachtclub': {'num_transactions': 1493,
                       'total_volume': 1036.0421504588755},
 'cool-cats-nft': {'num_transactions': 3000,
                   'total_volume': 1348.5594691153021},
 'cryptopunks': {'num_transactions': 112, 'total_volume': 50.751900000000006},
 'evolved-apes-inc': {'num_transactions': 2968,
                      'total_volume': 359.1697327089671},
 'neo-tokyo-identities': {'num_transactions': 541,
                          'total_volume': 5044.550201880792},
 'oraclenft': {'num_transactions': 465, 'total_volume': 20.55965},
 'veefriends': {'num_transactions': 86, 'total_volume': 204.13840133525068}}


In [7]:
print(slug_price)

{'boredapeyachtclub': {'total_volume': 1036.0421504588755, 'num_transactions': 1493}, 'neo-tokyo-identities': {'total_volume': 5044.550201880792, 'num_transactions': 541}, 'cryptopunks': {'total_volume': 50.751900000000006, 'num_transactions': 112}, 'cool-cats-nft': {'total_volume': 1348.5594691153021, 'num_transactions': 3000}, 'evolved-apes-inc': {'total_volume': 359.1697327089671, 'num_transactions': 2968}, 'oraclenft': {'total_volume': 20.55965, 'num_transactions': 465}, 'veefriends': {'total_volume': 204.13840133525068, 'num_transactions': 86}}
